<a href="https://colab.research.google.com/github/muhammedsanjidkm/Onset-date-determination-using-liebmann-method/blob/main/filtering%20mosdac%20%20hdf%20files%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import h5py
import numpy as np
import os
from datetime import datetime
from collections import defaultdict
from tabulate import tabulate

# Specify the folder containing the HDF5 files(ONE YEAR )
folder_path = "D:/sanjid books/second project/1.1/"

# Function to extract AOD values for a given file
def extract_aod(file_path):
    filename_parts = os.path.basename(file_path).split('_')#splitting file name to find date
    date_str = filename_parts[1]  # This extracts "01APR2023"
    date = np.datetime64(datetime.strptime(date_str, '%d%b%Y').date(), 'D')  # Parse date correctly

    with h5py.File(file_path, 'r') as file:
        aod = file['AOD'][:]# extracting AOD
        aod[aod == -999]=0   # Replace -999 with 0 or NaN
        lat = file['latitude'][:]  # latitude
        lon = file['longitude'][:]  # longitude

        # Create meshgrid for latitude and longitude
        lat_grid, lon_grid = np.meshgrid(lat, lon, indexing='ij')

        # Flatten latitude, longitude, and AOD arrays for easy manipulation
        lat_flat = lat_grid.flatten()
        lon_flat = lon_grid.flatten()
        aod_flat = aod.flatten()

        # Create a boolean mask for the specified coordinate range ( 20km-20km over kanpur )
        mask = ((lat_flat >= 26.42291) & (lat_flat <= 26.60309) & (lon_flat >= 80.1313) & (lon_flat <= 80.3327))

        # Filter AOD values based on the mask
        filtered_lat = lat_flat[mask]
        filtered_lon = lon_flat[mask]
        filtered_aod = aod_flat[mask]


        return date, filtered_aod

# Initialize a dictionary to store AOD values grouped by date
aod_dict = defaultdict(list)

# Iterate over each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".h5"):
        file_path = os.path.join(folder_path, filename)
        date, filtered_aod = extract_aod(file_path)
        aod_dict[date].extend(filtered_aod)

# Calculate the mean AOD for each day
mean_aod_by_day = {}
for date, aod_list in aod_dict.items():
    mean_aod_by_day[date] = np.mean(aod_list)

# Create a list to store the summary table data
summary_table_data = []

# Combine date and mean AOD into summary table data
for date, mean_aod in sorted(mean_aod_by_day.items()):
    summary_table_data.append([date, mean_aod])

# Print the summary table
print(tabulate(summary_table_data, headers=["Date", "Mean AOD"], tablefmt="grid"))


FileNotFoundError: [Errno 2] No such file or directory: 'D:/sanjid books/second project/1.1/'